## Reading and writing from files

The `open` function is the standard way to gain access to a file on your filesystem.  Files can be opened for reading only, writing, or both reading and writing.  By default files are opened in read only mode.  The primary argument to the `open` function is a string giving the path and name of the file you want to open:

In [1]:
### open function and file objects

## You can download `covid-ref.fsa` from the course repository at 
##    https://github.com/Bio724D/Bio724D_2023_2024/tree/main/data
f = open("covid-ref.fsa")

The simplest way to manipulate a file is simply to read all the information from it, return the data in the file as a string:

In [2]:
### reading everything from a file as a single string
s = f.read()

Once you've read what you need from the file it's good practice to close it (failing to close a file can lead to a memory leak in some contexts, but it's usually not a problem in an interactive environment like Jupyter notebooks).

In [3]:
f.close()

An alternate way to read a file is within a `with` statement as illustrated below.  The advantage of the `with` function is it insures you're file is closed (i.e. you don't need to explicitly call the `close` method).

In [5]:
with open("covid-ref.fsa") as f:
    s = f.read()

Once we've read the file into a string we can apply all the standard string methods and operators to it:

In [6]:
len(s)  ## how many characters were in the file?

30429

In [7]:
s[:50]  ## first 50 chars in the file

'>NC_045512.2 Severe acute respiratory syndrome cor'

## Reading a file by lines

Sometimes it's more convenient or more efficient to get the information in the file in terms of lines.  The `readlines` method associated with file object read's all the lines at once in a list:

In [7]:
## return a list of the lines in the file
with open("./covid-ref.fsa") as f:
    lines = f.readlines()

In [8]:
len(lines)

430

In [9]:
lines[0]

'>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome\n'

In [10]:
lines[1]

'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA\n'

## Reading a file one line at a time

The `readlines` function illustrated above reads all the lines at once. That's works well if your file has a modest number of lines, but for a file with millions of lines (or very long single lines)  `readlines` might exhaust the memory of your computer.  One way to work around this is to process files line by line, reading only one line at a time.  This can be done with a for loop applied directly to the file object:

In [11]:
with open("./covid-ref.fsa") as f:
    nchars = []
    for line in f:
        nchars.append(len(line)) # let's pretend our task was to count line lengths for files w/millions of lines

nchars[:10]  # the first chars 

[97, 71, 71, 71, 71, 71, 71, 71, 71, 71]

For such a simple computation we'd typically use a list comprehension:

In [12]:
with open("./covid-ref.fsa") as f:
    nchars = [len(line) for line in f]

nchars[:10] 

[97, 71, 71, 71, 71, 71, 71, 71, 71, 71]

# The FASTA file format for sequence data

The FASTA file format is the most commonly used file format used to represent nucleotide and protein sequence data.  Wikipedia has a good [overview of the FASTA format](https://en.wikipedia.org/wiki/FASTA_format).  

Summary of FASTA format:
 
 * Each file can hold one or more sequence records
 
 * The beginning of each record is delimited by a line called a header, which has a `>` character at the beginning, followed by the name associated with that record (and an optional description). For example `>seq1 Involved in...` would indicate the beginning of a record with the name `seq1` and the description "Involved in...".
 
 * On or more sequence lines follow header lines.  These lines are usually wrapped to have length <=80 characters but this is not required.



Below is a simple function that will parse a FASTA file, returning each record as an element in a Python dictionary with the first word in the header line of each record as the key. 

This implementation isn't particularly robust or optimal, but illustrates some key aspects of flow-control in Python and parsing non-tabular data.


In [8]:
def parse_FASTA(fname):
    record_dict = {}
    f = open(fname, 'r')
    
    recname = ""           # will hold names of records
    seq = ""               # will hold seq strings
    active_record = False  # indicates whether we are currently working on building a record
    
    for line in f.readlines():
        
        line = line.strip()  # strip any whitespace at beginning/end of line
        
        if line == "":       # empty line
            continue         # go to next iteration of for loop
            
 
        if line[0] == ">":                 # are we dealing with a new record?
            if active_record:              # did we already have an active record?
                record_dict[recname] = seq # if so, add to old active record to the dict so we can
                                           # begin a new one 
            
            recname = line[1:].split()[0]  # name of new record
            seq = ""                       # reset variable holding the string
            active_record = True           # set flag to indicate we now have an active record
            continue                       # go to the next iteration of for loop, as there's nothing else to do
        
        seq += line
        
    if active_record:               # if we've exhausted all the lines, we might still have an active record
        record_dict[recname] = seq  # if so, add it to the dict

    f.close() # remember to close the file!

    return record_dict
        
            

To test our `parse_FASTA` function download the [`Spike-protein-aligned.fasta`](https://github.com/Bio724D/Bio724D_2023_2024/tree/main/data/Spike-protein-aligned.fasta) file  above to your computer and modify the lines below to load and parse the sequence records contained in that file.

In [9]:
recs = parse_FASTA("./Spike-protein-aligned.fasta")

In [10]:
type(recs)

dict

In [11]:
len(recs)

7

In [12]:
list(recs.keys())  # get a list of the keys in recs

['KF367457.1',
 'AY278488.2',
 'NC_004718.3',
 'MG772933.1',
 'MT040333.1',
 'MN908947.3',
 'MN996532.1']

In [13]:
recs["MN996532.1"][:10]  # get the first 10 characters in the sequence for the record with this name

'M-FVFLVL-L'

In [15]:
# print the first 25 positions in the alignment for all the recrods
for key in recs.keys():
    print(recs[key][:25])

MKLLVLVF-ATLVSSYTIEKCLDFD
M-FIFLLF-LTLTSGSDLDRCTTFD
M-FIFLLF-LTLTSGSDLDRCTTFD
M-LFFLFLQFALVNS----QCVNLT
M-FVFLFV-LPLVSS----QCVNLT
M-FVFLVL-LPLVSS----QCVNLT
M-FVFLVL-LPLVSS----QCVNLT


In [ ]:
spikes = parse_FASTA("./spi")